## 1. 数据集导入和数据预处理

### 加载 csv 文件

* pandas.read_csv(): 用于加载CSV格式的数据文件，分别加载训练数据 (train.csv) 和测试数据 (test.csv)。
* info() 和 head(): info() 显示数据的基本信息（例如列名、数据类型等），head() 显示数据的前几行内容，用于快速预览数据。

In [4]:
import pandas

train_file_path = './data/train.csv'
test_file_path = './data/test.csv'
train_data = pandas.read_csv(train_file_path) # 加载 CSV 文件
test_data = pandas.read_csv(test_file_path)

# 显示数据的前几行以及基本信息
train_data_info = train_data.info()
train_data_head = train_data.head()

train_data_head

print('------------------------------------------------------------------------')

test_data_info = test_data.info()
test_data_head = test_data.head()

test_data_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       333 non-null    int64  
 1   crim     333 non-null    float64
 2   zn       333 non-null    float64
 3   indus    333 non-null    float64
 4   chas     333 non-null    int64  
 5   nox      333 non-null    float64
 6   rm       333 non-null    float64
 7   age      333 non-null    float64
 8   dis      333 non-null    float64
 9   rad      333 non-null    int64  
 10  tax      333 non-null    int64  
 11  ptratio  333 non-null    float64
 12  black    333 non-null    float64
 13  lstat    333 non-null    float64
 14  medv     333 non-null    float64
dtypes: float64(11), int64(4)
memory usage: 39.1 KB
------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 14 columns):
 #   Column   Non-N

,ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
1,6,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21
2,8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,396.90,19.15
3,9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93
4,10,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10


### 数据预处理

* 特征和目标变量的分离：我们从数据集中移除了房价的目标列 medv 和 ID 列，特征用于模型的训练和预测，而 medv 作为目标变量。
* StandardScaler：对特征进行标准化（归一化），即将每个特征缩放到均值为0，标准差为1的范围。这对于模型训练非常重要，因为不同尺度的特征会影响模型的学习效果。


In [5]:
from sklearn.preprocessing import StandardScaler # 用于数据预处理

# 分离特征和目标变量
X_train = train_data.drop(columns=['ID', 'medv'])  # 特征
y_train = train_data['medv']  # 目标变量

X_test = test_data.drop(columns=['ID'])

y_test_path = './data/submission_example.csv'
y_test_data = pandas.read_csv(y_test_path)
y_test = y_test_data['medv']

# 标准化特征
scaler = StandardScaler() # 标准化特征通过删除均值并缩放到单位方差。
X_train_scaled = scaler.fit_transform(X_train) # 将数据拟合，然后转换它。将变换器拟合到 X 和 y
X_test_scaled = scaler.transform(X_test) # 按照中心化和缩放进行标准化。

# X_train_scaled[:5]
# y_train[:5]  # 查看前5个样本的数据处理结果

### 转换数据为Tensor格式

* torch.tensor(): 将标准化后的训练和测试数据转换为 PyTorch 的张量格式，以便能够用于神经网络模型的训练和推断。
* view(-1, 1): 将目标变量 y_train 和 y_test 的形状调整为 (样本数, 1)，因为我们是在做回归问题，每个样本的输出是一个单一的房价预测值。

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

# 转换数据为Tensor格式
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

## 2. 构建神经网络模型

* class BostonHousingModel(nn.Module): 定义一个继承自 nn.Module 的神经网络类，用于构建房价预测模型。
* fc1, fc2, fc3: 分别为三层全连接层。第一层接收13个输入特征并输出64个隐藏节点，第二层输出32个节点，最后一层输出一个预测房价的标量。
* torch.relu(): ReLU激活函数，定义每个隐藏层的非线性激活，使网络能够学习复杂的非线性关系。



In [7]:
class BostonHousingModel(nn.Module):
    def __init__(self):
        super(BostonHousingModel, self).__init__()
        self.fc1 = nn.Linear(13, 64)  # 输入13个特征，输出64个节点
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)   # 输出一个房价预测值

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 实例化模型
model = BostonHousingModel()


## 3. 损失函数和优化器

* loss_fn = nn.MSELoss(): 定义损失函数为均方误差（MSE），MSE是回归任务中的常用损失函数，用于衡量预测值与实际值的差异。
* optimizer = optim.Adam(): 定义优化器为 Adam，Adam 是一种优化算法，它在大多数深度学习任务中表现优异。

In [8]:
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 3. 训练模型

* model.train(): 将模型设置为训练模式，这对于一些特定层如Dropout是有影响的。
* optimizer.zero_grad(): 清空梯度缓存（PyTorch默认梯度是累积的）。
* predictions = model(X_train_tensor): 前向传播计算模型的预测值。
* loss.backward(): 反向传播计算每个参数的梯度。
* optimizer.step(): 使用Adam优化器根据梯度更新模型参数。
* 每20个epoch输出一次损失值，以便跟踪训练过程。


In [9]:
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()  # 清除之前的梯度
    predictions = model(X_train_tensor)
    loss = loss_fn(predictions, y_train_tensor)
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型参数

    # 每隔20个epoch输出一次损失
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 606.095458984375
Epoch 20, Loss: 577.7645263671875
Epoch 40, Loss: 529.2919311523438
Epoch 60, Loss: 431.3033142089844
Epoch 80, Loss: 278.24945068359375
Epoch 100, Loss: 142.37112426757812
Epoch 120, Loss: 88.0057601928711
Epoch 140, Loss: 60.16559982299805
Epoch 160, Loss: 43.44068908691406
Epoch 180, Loss: 34.29367446899414


## 4. 模型训练完成，接下来将在测试集上评估模型性能

* model.eval(): 将模型设置为评估模式，这会影响诸如 Dropout 等层的行为。
* with torch.no_grad(): 在评估时禁用梯度计算，以节省内存和计算资源。
* 通过前向传播计算测试集上的预测结果，并计算测试集上的损失 test_loss。


In [10]:
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_loss = loss_fn(test_predictions, y_test_tensor)

print(f"Test Loss: {test_loss.item()}")

Test Loss: 78.98806762695312
